In [ ]:
!pip install wget
import wget
import os
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
  except RuntimeError as e:
    print(e)
print('GPUs:', gpus)

# Daten laden und präparieren

Wir betrachten einen Datensatz mit Hunden und Katzen.

In [ ]:
classes = ['Cat', 'Dog']

In [ ]:
train_path = 'data/PetImages/train'
valid_path = 'data/PetImages/valid'
test_path = 'data/PetImages/test'

Nun erzeugen wir aus den Ordnern die zugehörigen Batches.
`flow_from_directory` erzeugt augmentierte Bilder.
In unserem Beispiel werden diese einfach auf die Größe `224x224` skaliert.

In [ ]:
image_size = 224
batch_size = 10

train_batches = ImageDataGenerator(rescale=1./255) \
    .flow_from_directory(directory=train_path,
                         target_size=(image_size, image_size),
                         classes=classes,
                         batch_size=batch_size)
valid_batches = ImageDataGenerator(rescale=1./255) \
    .flow_from_directory(directory=valid_path,
                         target_size=(image_size, image_size),
                         classes=classes,
                         batch_size=batch_size)
test_batches = ImageDataGenerator(rescale=1./255) \
    .flow_from_directory(directory=test_path,
                         target_size=(image_size, image_size),
                         classes=classes,
                         batch_size=batch_size, shuffle=False)

In [ ]:
assert train_batches.n == 2*500
assert valid_batches.n == 2*100
assert test_batches.n == 2*50
assert train_batches.num_classes == valid_batches.num_classes == test_batches.num_classes == 2

In [ ]:
import matplotlib.pyplot as plt

def plot_images(images_arr):
    fig, axes = plt.subplots(1, 10, figsize=(20, 20))
    axes = axes.flatten()
    for img, ax in zip(images_arr, axes):
        ax.imshow(img)
        ax.axis('off')
    plt.tight_layout()
    plt.show()

Sehen wir uns das erste Trainingsbatch kurz an.

In [ ]:
imgs, labels = next(train_batches)

plot_images(imgs)
print(labels)

# Neuronales Netz erstellen

Nun kommen wir endlich zu unserem neuronalen Netz.
Wir verwenden ein Convolutional Neural Network, das durch `tf.keras.models.Sequential`
realisiert und verwendet zwei `tf.keras.layers.Conv2D`-Ebenen, zwei `tf.keras.layers.MaxPool2D`-Ebenen
und zum Schluss noch ein Fully-connected Layer, welches wir durch `tf.keras.layers.Flatten`
und `tf.keras.layers.Dense` bekommen.

<img src="assets/cnn.svg"/>

In [ ]:
model = Sequential([
    Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same', input_shape=(image_size, image_size, 3)),
    MaxPool2D(pool_size=(2,2), strides=2),
    Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
    MaxPool2D(pool_size=(2,2), strides=2),
    Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),

    Flatten(),
    Dense(units=2, activation='softmax'),
])
model.summary()

# Neuronales Netz trainieren

Für unsere Loss-Funktion verwenden wir Categorical Crossentropy
und für den Optimierer verwenden wir `tf.keras.optimizers.Adam`,
da dieser bessere Ergebnisse als Stochastic Gradient Descent liefert.

In [ ]:
model.compile(optimizer=Adam(lr=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

Mit `fit()` trainieren wir das neuronale Netz.

In [ ]:
model.fit(x=train_batches, validation_data=valid_batches, epochs=3, verbose=2)

# Vorhersagen auf Testdatensatz

Betrachten wir nun das Test-Set.
Da das Test-Set nicht gemischt wurde,
sind die ersten Bilder alles Katzen.

In [ ]:
test_imgs, test_labels = next(test_batches)
plot_images(test_imgs)
print(test_labels)

Nun sagen wir das Test-Set mithilfe unseres trainierten Netzes mit `predict()` vor.

In [ ]:
predictions = model.predict(x=test_batches, verbose=0)

In [ ]:
import numpy as np
preds = np.argmax(predictions, axis=1)
print("Predictions:\n", preds)
print("Actual:\n", test_batches.classes)
print("Accuracy:", sum(preds == test_batches.classes)/preds.shape[0])

In [ ]:
from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt

def plot_cm(y_true, y_pred, classes, title='Konfusionsmatrix', cmap=plt.cm.Blues):
    cm = confusion_matrix(y_true=y_true, y_pred=y_pred)
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    thresh = cm.max() * 2. / 3
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
            horizontalalignment="center",
            color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('Tatsächlich')
    plt.xlabel('Vorhersage')

In [ ]:
plot_cm(test_batches.classes, preds, classes)